In [ ]:
%run init_model.py 'algo_daily_breakout'

# Step 1) Data Preparation

In [ ]:
# get S3 bucket
s3bucket=!(aws s3 ls | grep algotrading- | awk  '{print $3}')
s3bucket=s3bucket[0]
s3bucket

In [ ]:
import sys
!{sys.executable} -m pip install PyAthena

In [ ]:
import os
import sagemaker as sage
from sagemaker import get_execution_role
import datetime
from sagemaker.tensorflow import TensorFlow
import json

role = get_execution_role()
sess = sage.Session()
region = sess.boto_session.region_name

In [ ]:
import pandas as pd
from pyathena import connect
conn = connect(s3_staging_dir='s3://'+s3bucket+'/results/',
               region_name=region)

df = pd.read_sql("SELECT dt,open,high,low,close,vol FROM algo_data.hist_data_daily;", conn)
df.set_index(pd.DatetimeIndex(df['dt']),inplace=True)
del df['dt']
df.head()

In [ ]:
trainCount=int(len(df)*0.4)
dfTrain = df.iloc[:trainCount]

dfTest = df.iloc[trainCount:]
dfTest.to_csv('local/'+algo_name+'/input/data/training/data.csv')
dfTest.head()

In [ ]:
%matplotlib notebook
dfTest["close"].plot()

# Step 2) Modify Strategy Configuration 

In the following cell, you can adjust the parameters for the strategy.

* `user` = Name for Leaderboard (optional)
* `go_long` = Go Long for Breakout (true or false)
* `go_short` = Go Short for Breakout (true or false)
* `period` = Length of window for previous high and low
* `size` = The number of shares for a transaction

`Tip`: A good starting point for improving the strategy is to lengthen the period of the previous high and low. Equity Markets tend to have a long bias and if you only consider long trades this might improve the performance.

In [ ]:
%%writefile local/{algo_name}/input/config/hyperparameters.json
{ "user" : "user",
  "go_long" : "true",
  "go_short" : "true",
  "period" : "50",
  "size" : "100"
}

In [ ]:
%run update_config.py $algo_name

# Step 3) Modify Strategy Code

`Tip`: A good starting point for improving the strategy is to add additional indicators like ATR (Average True Range) before placing a trade. You want to avoid false signals if there is not enough volatility.

Here are some helpful links:
* Backtrader Documentation: https://www.backtrader.com/docu/strategy/
* TA-Lib Indicator Reference: https://www.backtrader.com/docu/talibindautoref/
* Backtrader Indicator Reference: https://www.backtrader.com/docu/indautoref/

In [ ]:
%%writefile model/{algo_name}.py
import backtrader as bt
from algo_base import *
import pytz
from pytz import timezone

class MyStrategy(StrategyTemplate):

    def __init__(self):  # Initiation
        super(MyStrategy, self).__init__()
        self.config["period"]=int(self.config["period"])
        self.config["size"]=int(self.config["size"])
        self.config["go_long"]=(str(self.config["go_long"]).lower()=="true")
        self.config["go_short"]=(str(self.config["go_short"]).lower()=="true")

        self.highest = bt.ind.Highest(period=self.config["period"])
        self.lowest = bt.ind.Lowest(period=self.config["period"])
        self.size = self.config["size"]
        
    def init_broker(broker):
        broker.setcash(100000.0)
        broker.setcommission(commission=0.0) 
        
    def add_data(cerebro):
        data = btfeeds.GenericCSVData(
            dataname=MyStrategy.TRAIN_FILE,
            dtformat=('%Y-%m-%d'),
            timeframe=bt.TimeFrame.Days,
            datetime=0,
            time=-1,
            high=2,
            low=3,
            open=1,
            close=4,
            volume=5,
            openinterest=-1
        )
        cerebro.adddata(data)
        
    def next(self):  # Processing
        super(MyStrategy, self).next()
        dt=self.datas[0].datetime.datetime(0)
        if not self.position:
            if self.config["go_long"] and self.datas[0] > self.highest[-1]:
                self.buy(size=self.size) # Go long
            elif self.config["go_short"] and self.datas[0] < self.lowest[-1]:
                self.sell(size=self.size) # Go short
        elif self.position.size>0 and self.datas[0] < self.highest[-1]:
            self.close()
        elif self.position.size<0 and self.datas[0] > self.lowest[-1]:          
            self.close()

# Step 4) Backtest Locally

**Please note that the initial docker build may take a few minutes. Subsequent runs are fast.**

In [ ]:
#Build Local Algo Image
!docker build -t $algo_name .
!docker run -v $(pwd)/local/$algo_name:/opt/ml --rm $algo_name train

In [ ]:
from IPython.display import Image
Image(filename='local/'+algo_name+'/model/chart.png')

## Refine your trading strategy (step 2 to 4). Once you are ready, move on to the next step.

# Step 5) Backtest on SageMaker and submit performance

In [ ]:
#Deploy Algo Image to ECS
!./build_and_push.sh $algo_name

In [ ]:
#Run Remote Forwardtest via SageMaker
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator 

role = get_execution_role()
sess = sage.Session()

WORK_DIRECTORY = 'local/'+algo_name+'/input/data/training'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix='data')
print(data_location)

conf_file='local/'+algo_name+'/input/config/hyperparameters.json'
with open(conf_file, 'r') as f:
    config = json.load(f)
#config['sim_data']='True'
print(config)

prefix=algo_name
job_name=prefix.replace('_','-')

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

algo = sage.estimator.Estimator(
    image_uri=image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
    base_job_name=job_name,
    hyperparameters=config,
    metric_definitions=[
        {
            "Name": "algo:pnl",
            "Regex": "Total PnL:(.*?)]"
        },
        {
            "Name": "algo:sharpe_ratio",
            "Regex": "Sharpe Ratio:(.*?),"
        }
    ])
algo.fit(data_location)

In [ ]:
#Get Algo Metrics
from sagemaker.analytics import TrainingJobAnalytics

latest_job_name = algo.latest_training_job.job_name
metrics_dataframe = TrainingJobAnalytics(training_job_name=latest_job_name).dataframe()
metrics_dataframe

In [ ]:
#Get Algo Chart from S3
model_name=algo.model_data.replace('s3://'+sess.default_bucket()+'/','')
import boto3
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sess.default_bucket())
my_bucket.download_file(model_name,'model.tar.gz')
!tar -xzf model.tar.gz
!rm model.tar.gz
from IPython.display import Image
Image(filename='chart.png') 

### Congratulations! You've completed this strategy.